In [3]:
import pandas as pd
import numpy as np
import funcs, gc, sqlite3

In [4]:
rats_unclassif = funcs.get_rats_unclassified()
rats_unclassif

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR,CIA


In [9]:
classificadores = funcs.get_df_classif()
rats_unclassif.loc[:,'SETOR'] = rats_unclassif.apply(lambda row: funcs.classifica_setor(row, classificadores), axis=1)
funcs.classifica_cia(rats_unclassif)

rats_unclassif

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [5]:
new_classif_dict = [
    {
        'nrat': unclass_list[0],
        'setor': unclass_list[-2],
        'cia': unclass_list[-1]
    }
    for unclass_list in rats_unclassif.values
]

new_classif_dict

[]

In [6]:
query_update = '''
UPDATE tbl_rat
    SET "SETOR" = ?,
    "CIA" == ?
WHERE
    "RAT.NUM_ATIVIDADE" = ?
'''

with sqlite3.connect('gdo.db') as conn:
    cursor = conn.cursor()
    for dict_class in new_classif_dict:
        cursor.execute(query_update, (dict_class['setor'], dict_class['cia'], dict_class['nrat']))
        conn.commit()
conn.close()